In [132]:
import torch
import numpy as np
import pickle
import pandas as pd
import plotly.express as px
import HMM_Likelyhood
import LogLikelihoodTorch
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import HMM_Likelyhood

with open('GMMsubsetData.pickle','rb') as f:
    X_tensor = pickle.load(f) 
torch.set_default_dtype(torch.float64)

labels = pd.read_csv("AALlabels.csv")
label=labels['labels'].values
Subjects=10
Soft=torch.nn.Softmax(0)

In [2]:
# Split train test first half train second half test
Xtrain = torch.zeros((90,165*10))
Xtest = torch.zeros((90,165*10))
for i in range(10):
    Xtrain[:,i*(165):((i+1)*165)] = X_tensor[:,i*(330):((i*330+165))]
    Xtest[:,i*(165):((i+1)*165)] = X_tensor[:,i*(330)+165:((i*330+330))]

# Number of Distributions in the model
K=10

# Initialize parameters for Mixture Model
Pi_MM,Kappa_MM,mu_MM = LogLikelihoodTorch.InitializeFF(Xtrain,90,K)

Parameters_MM = [
{'params':Pi_MM},
{'params':Kappa_MM},
{'params':mu_MM},
]

# Intilize optimizer and fit parameters for MM
AdamMM = torch.optim.Adam(Parameters_MM,lr=0.8)
AdamMM.zero_grad()
LogLikelihoodTorch.Optimizationloop(Xtrain,[Pi_MM,Kappa_MM,mu_MM],lose=LogLikelihoodTorch.log_likelihood,Optimizer=AdamMM,n_iters=2000,K=K)

# Intialize parameters for 
Kappa_HMM,mu_HMM,Tk_HMM,Pinit_HMM = HMM_Likelyhood.InitializeParametersFF(Xtrain,10,90,K)
Parameters_HMM= [
{'params':Pinit_HMM},
{'params':Kappa_HMM},
{'params':mu_HMM},
{'params':Tk_HMM},
]

# Initialize optimzer and fit parameters for HMM
AdamHMM=torch.optim.Adam(Parameters_HMM,lr=1.2)
AdamHMM.zero_grad()
HMM_Likelyhood.Optimizationloop(Xtrain,[Pinit_HMM,Kappa_HMM,mu_HMM,Tk_HMM],lose=HMM_Likelyhood.Accumulated_HHM_LLHalf,Optimizer=AdamHMM,n=10,n_iters=2000,K=K)

epoch 1; Log-Likelihood = -132950.6880288368
epoch 101; Log-Likelihood = -160474.5768187429
epoch 201; Log-Likelihood = -164049.4785055761
epoch 301; Log-Likelihood = -166665.52909914684
epoch 401; Log-Likelihood = -169086.15111770935
epoch 501; Log-Likelihood = -174066.74305226273
epoch 601; Log-Likelihood = -179840.4917110799
epoch 701; Log-Likelihood = -185246.98457890723
epoch 801; Log-Likelihood = -190617.85402122803
epoch 901; Log-Likelihood = -196000.9733514118
epoch 1001; Log-Likelihood = -201370.10660989035
epoch 1101; Log-Likelihood = -205468.45347101684
epoch 1201; Log-Likelihood = -209215.69301940477
epoch 1301; Log-Likelihood = -212584.70451365563
epoch 1401; Log-Likelihood = -215322.91196614795
epoch 1501; Log-Likelihood = -217271.03146948974
epoch 1601; Log-Likelihood = -218537.33428941414
epoch 1701; Log-Likelihood = -219411.1618667912
epoch 1801; Log-Likelihood = -220303.93623110303
epoch 1901; Log-Likelihood = -221043.04500247916
epoch 1; Log-Likelihood = -120712.7165

[tensor([[ -9.9984,  -1.2198,  -8.0130,  -1.4897,  -2.7290, -11.5675,  -3.2382,
           -3.8736,  -7.7120,  -8.9083],
         [ -8.9774, -11.1148,  -9.1192, -10.7266, -11.6772,  -4.0508, -11.1531,
          -10.5948,  -7.8741, -22.3167],
         [ -9.9781,  22.0798,  32.6489,  14.5656,  12.8497, -11.5676,  17.7225,
           -6.5661,  27.3585,  66.9212],
         [ -8.3258, -10.8022, -12.1886, -11.4048, -12.0255, -10.9980, -11.1619,
          -11.5175, -11.4567,  -5.3071],
         [ -8.7762, -10.4608,  -7.9828, -11.3265, -11.8153, -12.4754, -10.8913,
           -7.2455, -11.1492,  -5.8770],
         [-28.5230,  -6.6956, -11.7218, -11.1671, -11.0754, -10.9766,  -8.4756,
          -10.3556,   1.0857,  -8.1234],
         [ 40.5755, -11.1726,  -8.2741, -11.4464, -11.6088,  14.6177, -11.1109,
           53.9940, -11.2801,  -9.6027],
         [ -5.2446, -10.9316, -24.4527,  -5.7465, -11.9132,  -6.3780, -11.1267,
          -16.9089, -12.0173,  -9.7087],
         [ -5.1836,  -7.5332, -1

In [217]:
# Likelihood of MM
LogLikelihoodTorch.log_likelihood(Xtest,Pi_MM,Kappa_MM,mu_MM,90,K)

tensor(220351.9080, grad_fn=<SumBackward0>)

In [218]:
# Likelihood of HMM
HMM_Likelyhood.Accumulated_HHM_LLHalf(Xtest,Pinit_HMM,Kappa_HMM,mu_HMM,Tk_HMM,10,90,K)

tensor(185074.4691, grad_fn=<SumBackward0>)

In [93]:
# Estimiate pdf of each sample for each model
B_MM = HMM_Likelyhood.Softmax(LogLikelihoodTorch.log_pdf(Xtest,mu_MM,Kappa_MM,90).T)
B_MM = B_MM/B_MM.sum(0)

B_HMM = HMM_Likelyhood.log_pdf(Xtest,mu_HMM,Kappa_HMM,90).T
B_HMM = B_HMM/B_HMM.sum(0)

Traj1_MM = np.argmax(B_MM[:,0:165].detach().numpy(),0)
States_MM = np.argmax(B_MM.detach().numpy(),0)

T1 = np.round(Soft(Tk_HMM[0]).detach().numpy(),2)
T2 = np.round(Soft(Tk_HMM[1]).detach().numpy(),2)

In [216]:
# Plot Transition matrix of each subject
for sub in range(Subjects):
    fig=px.imshow(np.round(Soft(Tk_HMM[sub]).detach().numpy(),2),text_auto=True,color_continuous_scale='viridis')
    fig.update_layout(title=f"Transition Matrix Subject {sub+1}",
    width=500,height=500)
    fig.show()

In [153]:
# Estimate Propabillites of each state for HMM model
Prop_HMM_sub = [torch.zeros((10,165)) for _ in range(Subjects)]

for sub in range(Subjects):
    Prop_HMM_sub[sub][:,0] = HMM_Likelyhood.Softmax((HMM_Likelyhood.Softmax(Pinit_HMM[:,sub])*B_HMM[:,165*sub]))

    T1 = HMM_Likelyhood.Softmax(Tk_HMM[sub])
    for i in range(1 ,165):
        Prop_HMM_sub[sub][:,i] = Soft((T1@Prop_HMM_sub[sub][:,i-1])*B_HMM[:,i+165*sub])

Traj = [np.argmax(Prop_HMM_sub[sub].detach().numpy(),0) for sub in range(Subjects)]

In [211]:
# Transformating trajectory to comparible states
ComparisonIndex = [(0,1),(1,6),(2,9),(3,4),(4,3),(5,2),(6,5),(7,7),(8,8),(9,0)]
Transformed_States_MM = np.zeros(165*10)
for idx,state in enumerate(States_MM):
    Transformed_States_MM[idx] = ComparisonIndex[state][1]

In [214]:
# Plot States of 2 first subjects for each model
Time = np.arange(330)*3/4
fig = make_subplots(rows=2, cols=1)

fig.add_trace(
    go.Scatter(
            x=Time,
            y=Transformed_States_MM[0:165],
            mode='lines+markers',
            name="Subject 1 MM State"
            ),
    
    row=1,col=1)

fig.add_trace(go.Scatter(
                x=Time,
                y=Traj[0],
                mode='lines+markers',
                name="Subject 1 HMM State"),
                1,1)

fig.add_trace(go.Scatter(
            x=Time,
            y=Transformed_States_MM[165:330],
            mode='lines+markers',
            name="Subject 1 MM State"),
            2,1)

fig.add_trace(go.Scatter(
            x=Time,
            y=Traj[1],
            mode='lines+markers',
            name="Subject 1 HMM State"),
            2,1)


fig.update_layout(height=800, width=1200,
                title_text=None,
                showlegend=True)
fig.show()

In [210]:
# Compare Centroids of each model
Pi_MM = Soft(Pi_MM)
ComparisonIndex = [(0,1),(1,6),(2,9),(3,4),(4,3),(5,2),(6,5),(7,7),(8,8),(9,0)]


for Index in ComparisonIndex:
    fig = make_subplots(rows=1, cols=2,
                        shared_yaxes=True,
                        subplot_titles=(f" kappa = {np.round(Kappa_HMM[Index[0]].detach().numpy(),1)} ",f" pi = {np.round(Pi_MM[Index[0]].detach().numpy(),3)}, kappa = {np.round(Kappa_MM[Index[0]].detach().numpy(),1)}"))

    fig.add_trace(go.Bar(
                y=label,
                x=mu_HMM[:,Index[1]].detach().numpy(),
                orientation='h'),
                1,1)

    fig.add_trace(go.Bar(
                y=label,
                x=mu_MM[:,Index[0]].detach().numpy(),
                orientation='h'),
                1,2)


    fig.update_layout(height=1500, width=800,
                    title_text=f"State {Index[1]}:           Hidden Markov Model                            Mixture Model",
                    showlegend=False)
    fig.show()